In [27]:
! pip install elasticsearch tika sentence-transformers numpy
! pip install pymupdf
! pip install ibm_cloud_sdk_core
! pip install tqdm
! pip install requests

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [28]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://11.0.0.145:9200/")

# Define index with embedding field
index_name = "rag_composite_embedding"

mapping = {
    "mappings": {
        "properties": {
            "file_name": {"type": "text"},
            "chunk_id": {"type": "integer"},
            "content": {"type": "text"},
            "embedding": {"type": "dense_vector", "dims": 384}  # Dimension of chosen embedding model
        }
    }
}

# Create index
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")


Index 'rag_composite_embedding' already exists.


In [29]:
import fitz  # This is PyMuPDF
from sentence_transformers import SentenceTransformer
import numpy as np

# Load embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Read and extract text from PDF
pdf_path = "/home/sandeep-cc/Documents/rag/documents/Elastic_NV_Annual-Report-Fiscal-Year-2023.pdf"
doc = fitz.open(pdf_path)

# Extract text from each page
pdf_text = "\n".join([page.get_text("text") for page in doc])


questions_string = [
    "How does Elastic's business model, which combines a paid Elastic-managed hosted service offering and proprietary self-managed software, contribute to its product-led growth strategy?",
    "Elastic has experienced rapid growth in its customer base and total revenue. How do subscription-based pricing and international expansion play a role in this growth?",
    "What are the key features and benefits of Elastic's solutions, such as Search, Observability, and Security, and how do they help organizations find insights and drive AI and machine learning use cases from large amounts of data?",
    "How does Elastic's engineering organization approach the development and enhancement of products, and what strategies does it employ to ensure rapid product releases and high-quality hires?",
    "The document mentions that Elastic's sales and marketing strategy focuses on making it easy for users to begin using their products and driving rapid adoption. Can you discuss the specific tactics Elastic uses to achieve this, and how it maintains user engagement after initial adoption?",
    "Elastic's technology platform consists of the Elastic Stack, solutions, and software that support various deployment alternatives. How do innovations and new capabilities in the Elastic Stack benefit the solutions, and how can customers customize and extend these solutions to fit their needs?",
    "How does Elastic's focus on Diversity, Equity, and Inclusion (DEI) contribute to creating balanced teams and fostering a sense of belonging among all Elasticians?",
    "Can you explain Elastic's approach to fair pay, including their use of local third-party market data and external experts to ensure compensation practices are bias-free?",
    "How does Elastic's Elastician Resource Groups (ERG) initiative support specific shared identities, interests, and affinity groups, and what tangible benefits do these groups provide to Elasticians?",
    "How does the company's reliance on timely and successful product development, market acceptance, and effective management of risks associated with new product releases impact the success of new product introductions?"
    "In what ways do privacy, data protection, and information security laws and regulations, both in the United States and internationally, pose compliance risks and uncertainties for the company, and how might these impact its business and results of operations?  "
    "How does the company's ability to increase the usage of its products depend on the development of new use cases, and what role does its developer community play in this process?  "
    "How might the change in the licensing model for Elasticsearch and Kibana impact the adoption and usage of these products, and what potential consequences could this have for Elastic's brand and product awareness, as well as its paying customer base?  "
    "Given the importance of the sales and marketing function for Elastic's growth and success, what steps is the company taking to ensure that it has sufficient resources and capabilities in these areas, and what challenges and risks does it face in this regard?  "
    "How might the loss of key personnel or an inability to attract and retain highly skilled employees impact Elastic's future performance, and what measures is the company taking to mitigate these risks?  "
    "How does the company's dependence on lead generation strategies, such as offering free use of some product features and free trials, impact its ability to grow revenue and attract paying customers? 17. What are the potential risks and challenges the company may face as it expands its international operations, including exposure to political and economic instability, regulatory requirements, and intellectual property protection?  "
    "How might changes in tax laws, such as the OECD's Pillar One and Pillar Two blueprints and the Minimum Tax Rate Act 2024, impact the company's income tax obligations and overall financial results?  "
    "How might the increasingly complex tax laws in various jurisdictions impact the company's financial results, and what measures are being taken to mitigate this risk?  "
    "Considering the company's reliance on channel partners and third parties for international sales, what specific steps are being taken to ensure compliance with anti-bribery, anti-corruption, and anti-money laundering laws?  "
    "Given the concentration of share ownership with insiders, how might this affect shareholder influence on corporate matters, and what potential implications could this have for other shareholders?  "
    "What are the potential consequences if the company is unable to generate sufficient cash to service its debt obligations?  "
    "How could the proposed Dutch dividend withholding tax and Alternative Withholding Tax impact the company's future dividend payments or share repurchases?  "
    "What are the risks and challenges associated with the company's acquisition strategy, and how might they affect the company's financial condition and results of operations?"  
    "How does Elastic's revenue generation model focus on product-led growth, and what role do subscription tiers and features play in this strategy?"  
    "Can you explain how Elastic's Net Expansion Rate is calculated and what it indicates about customer relationships and expansion over time?  "
    "How has the COVID-19 pandemic impacted Elastic's business, and what measures has the company taken to navigate the current economic environment?  "
    "How did the significant increase in the provision for income taxes impact Elastic N.V.'s effective tax rate for the year ended April 30, 2023, and what factors contributed to this increase?  "
    "Considering the company's historical operating losses and potential future capital requirements, how confident are you that Elastic N.V.'s existing cash, cash equivalents, and marketable securities will be sufficient to fund their operating and capital needs for at least the next 12 months?  "
    "Given the critical audit matter related to revenue recognition, specifically the identification and evaluation of terms and conditions in contracts, how do you ensure that management's judgments in this area are reasonable and appropriate, and what steps have been taken to address this matter during the audit process?  "
]



# Split into chunks (e.g., by sentence or paragraphs)
chunk_size = 500  # Adjust as needed
chunks = [pdf_text[i : i + chunk_size] for i in range(0, len(pdf_text), chunk_size)]







In [30]:

# Compute question embedding once (assuming it's the same for all chunks)
questions_embedding = model.encode(questions_string).tolist()[0]  # Extract first row if it's a batch

# Process and store chunks
bulk_data = []  # For bulk indexing in Elasticsearch



In [11]:
for chunk_id, chunk in enumerate(chunks):
    text_embedding = model.encode(chunk).tolist()  # Compute only once

In [12]:

# Compute composite embedding
composite_embedding = [
    0.7 * t + 0.3 * q for t, q in zip(text_embedding, questions_embedding)
]



In [13]:



for chunk_id, chunk in enumerate(chunks):

    # Prepare document for Elasticsearch
    document = {
        "file_name": pdf_path.split("/")[-1],
        "chunk_id": chunk_id,
        "content": chunk,
        "text_embedding": text_embedding,  # Store original
        "composite_embedding": composite_embedding,  # Store weighted sum
    }
    bulk_data.append({"index": {"_index": index_name, "_id": chunk_id}})
    bulk_data.append(document)

# Bulk upload instead of indexing individually
es.bulk(index=index_name, body=bulk_data)

ObjectApiResponse({'errors': False, 'took': 400, 'items': [{'index': {'_index': 'rag_composite_embedding', '_id': '0', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rag_composite_embedding', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rag_composite_embedding', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rag_composite_embedding', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rag_composite_embedding', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful':

In [40]:
from scipy.spatial import distance
query_text = "total revennue of 2023?"
query_embedding = model.encode(query_text).tolist()

# query = {
#     "size": 3,  # Retrieve top 3 most relevant chunks
#     "query": {
#         "script_score": {
#             "query": {"match_all": {}},
#             "script": {
#                 "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
#                 "params": {"query_vector": query_embedding},
#             },
#         }
#     },
# }

# query = {
#     "size": 3,  # Retrieve top 3 most relevant chunks
#     "query": {
#         "bool": {
#             "should": [
#                 {  # BM25 (Lexical Search)
#                     "match": {"content": query_text}
#                 },
#                 {  # Semantic Search (Vector Search)
#                     "script_score": {
#                         "query": {"match_all": {}},
#                         "script": {
#                             "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
#                             "params": {"query_vector": query_embedding},
#                         },
#                     }
#                 }
#             ],
#             "minimum_should_match": 1  # At least one method must match
#         }
#     }
# }

# query = {
#     "size": 3,
#     "query": {
#         "script_score": {
#             "query": {"match_all": {}},
#             "script": {
#                 "source": "cosineSimilarity(params.query_vector, 'text_embedding') + 1.0",
#                 "params": {"query_vector": query_embedding}
#             }
#         }
#     }
# }
# query = {"query": {"match": {"content": query_text}}}
# response = es.search(index="rag_composite_embedding", body=query)




# response = es.search(index="rag_composite_embedding", body=query)
# print(response)

# retrieved_chunks = [hit["_source"]["chunk"] for hit in response["hits"]["hits"]]
# context = "\n\n".join(retrieved_chunks)
# retrieved_chunks = [hit["_source"]["content"] for hit in response["hits"]["hits"]]
# context = "\n\n".join(retrieved_chunks)

# print("Retrieved Context:\n", context)


query = {
    "size": 3,  # Retrieve top 3 relevant chunks
    "query": {
        "bool": {
            "should": [
                {  # BM25 Lexical Search
                    "match": {"content": query_text}
                },
                {  # Semantic Search (Vector Search)
                    "script_score": {
                        "query": {"match_all": {}},
                        "script": {
                            "source": "cosineSimilarity(params.query_vector, 'text_embedding') + 1.0",
                            "params": {"query_vector": query_embedding}
                        }
                    }
                }
            ],
            "minimum_should_match": 1  # At least one method must match
        }
    }
}

response = es.search(index="rag_composite_embedding", body=query)

# Retrieve results
retrieved_chunks = [hit["_source"]["content"] for hit in response["hits"]["hits"]]
context = "\n\n".join(retrieved_chunks)

print("Retrieved Context:\n", context)



Retrieved Context:
  thousands):
Year Ended April 30,
2023
2022
2021
Amount
% of
Total
Revenue
Amount
% of
Total
Revenue
Amount
% of
Total
Revenue
Elastic Cloud
$ 424,053 
 40 % $ 298,615 
 35 % $ 
166,319 
 27 %
Other subscription
560,709 
 52 %
500,155 
 58 %
401,020 
 66 %
Total subscription
984,762 
 92 %
798,770 
 93 %
567,339 
 93 %
Services
84,227 
 8 %
63,604 
 7 %
41,150 
 7 %
Total revenue
$ 1,068,989 
 100 % $ 862,374 
 100 % $ 
608,489 
 100 %
For the years ended April 30, 2023 and 2022, license revenue

mes
2,872 
2,263 
609 
1.4
Total
$ 
92,600 $ 
46,795 $ 
45,805 
3.2
Foreign currency translation adjustment
(5) 
Total
$ 
45,800 
86

Amortization expense for the intangible assets for the years ended April 30, 2023, 2022, and 2021 was as follows (in 
thousands):
Year Ended April 30,
2023
2022
2021
Cost of revenue – subscription
$ 
11,781 $ 
10,503 $ 
8,437 
Sales and marketing
4,887 
5,280 
5,730 
Total amortization of acquired intangible assets
$ 
16,668 $ 
15,783 $ 
14,1

In [41]:

import logging
import requests
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

API_TOKEN_IBM = "y90rAx1FDO0NEBsp51_4F9VEjCsNH_Xm0-bbLY65eDQ-"
PROJECT_ID_IBM = "21cfeb25-cf88-4099-95d4-db601d5c9730"
authenticator = IAMAuthenticator(API_TOKEN_IBM)
service = "Bearer " + authenticator.token_manager.get_token()

url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
# Define the system role and user prompt for Mistral
# prompt = '''
# [INST] You are an intelligent assistant specializing in answering questions based on given information. 
# Use the provided context to generate a clear, concise, and well-structured response. If the context does not contain the answer, state that explicitly instead of guessing.

# ### Context:
# {context}

# ### Question:
# Who audits Elastic?

# ### Answer:
# [/INST]
# '''


prompt = f'''
[INST] You are an intelligent assistant specializing in answering questions based on given information. 
Use the provided context to generate a clear, concise, and well-structured response. 
If the context does not contain the answer, state that explicitly instead of guessing.

### Context:
{context}

### Question:
{query_text}

### Answer:
[/INST]
'''



# prompt=truncate_text(prompt,7500)
body = {
    "input":prompt ,
    "parameters": {
        "decoding_method": "greedy",
        "max_new_tokens": 1000,
        "stop_sequences": [],
        "repetition_penalty": 1
    },
    # "model_id": "ibm/granite-13b-chat-v2",
    "model_id": "mistralai/mixtral-8x7b-instruct-v01",
    "project_id": PROJECT_ID_IBM,
    "moderations": {
        "hap": {
            "input": {"enabled": False},  # Disable input moderation
            "output": {"enabled": False}  # Disable output moderation
        }
    }
}

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": service
}

response = requests.post(
    url,
    verify=False,
    headers=headers,
    json=body
)

if response.status_code != 200:
    logging.warning(f"Status Code --> {response.status_code}")
    print("model context window exceeded for this document"+str(response.text))
data = response.json()
print(data['results'][0]['generated_text'])

/home/sandeep-cc/Documents/rag/env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'us-south.ml.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


The total revenue for the year ended April 30, 2023 was $1,068,989 thousand.
